# Implementing an SMC Likelihood Annealing Sampler

This notebook describes how to set up a SMC sampler from Algorithm 1. This SMC sampler assumes that the user will provide the set of temperatures used in likelihood annealing.

$$
\pi_t(\theta|y) \propto f(y|\theta)^{\gamma_t}\pi(\theta) \quad \text{for } t=1,\ldots,T,
$$

The user will specify the values $\gamma = (0, ..., 1)$ that define the sequence, the likelihood $f(y|\theta)$, prior $\pi$. 

## The Algorithm

Input: $\gamma = (\gamma_1=0,\gamma_2,...,\gamma_T=1)$ and random walk covariance $\Sigma$, number of MCMC move steps $R$.

1. Sample initial $\xi^i_1\sim \pi_1(\theta)$ and set $W_1^i = \frac{1}{N}$ for $i\in\{1,...,N\}$
2. For each time t = 2, ..., T

    (i) Calculate weights (Importance sampling)
            $$\tilde{w}_{t}^i = W_{t-1}^if(y|\theta = \xi^i_{t-1})^{\gamma_t - \gamma_{t-1}} \qquad W_{t}^i = \frac{\tilde{w}_{t}^i}{\sum_{j=1}^N\tilde{w}_{t}^j},
            $$
            for $i=1,\ldots,N.$

    (ii)  Resample $A_t^i$ ~ Cat $(W_{t}^1,...,W_{t}^N)$ and set $W_t^i=1/N$ for $i\in \{1,...,N\}$
    
    (iii) Diversify (MCMC) with $R$ RW proposals with covariance $\Sigma$
            $$
            \xi_{t}^i \sim K_{\pi_t}(\cdot \mid \theta = \xi_{t-1}^{A_{t}^i}) , \quad \text{  for  } i=1,\ldots,N.
            $$

### Coding utilities 

For numerical stability calculations will be done on the log scale - i.e. the user will specify the log likelihood and log prior. We also need the following function which will perform a numerically stable log sum of exponential numbers. We also need some necessary packages.


In [ ]:
function logsumexp(x)
    my_max = maximum(x)
    x_new = x .- my_max
    return my_max + log(sum(exp.(x_new)))
end

using Distributions, StatsBase, LinearAlgebra

### Coding the algorithm

The user will specify a function to simulate from the prior, evaluate the log prior, log likelihood, the choice of $\gamma$, $\Sigma$ and $R$.

In [ ]:
function smc_sampler_lanneal(N, simprior, logprior, loglike; γ = range(0, stop=1, length=15).^5, Σ = nothing, R=20)

    # Sample initial from prior and set weights
    ξ = simprior(N)
    log_W = -log(N)*ones(N)

    d = size(ξ, 2) # Get dimension of θ

    # If RW sigma is not specified use a default
    if isnothing(Σ)
        Σ = cov(ξ)
    end

    loglike_curr = fill(NaN, N)
    logprior_curr = fill(NaN, N)
    for i in 1:N
        loglike_curr[i] = loglike(ξ[i, :])
        logprior_curr[i] = logprior(ξ[i, :])
    end
  
    for t in 2:length(γ)

        # Calcuate (log) weights w and normalised weights W
        log_w = log_W .+ (γ[t] - γ[t-1]) .* loglike_curr
        log_W .= log_w .- logsumexp(log_w) 
        
        # Resample the particles and reset the weights to 1/N
        inds = sample(1:N, Weights(exp.(log_W)), N) # Multinomial

        ξ = ξ[inds, :]
        loglike_curr = loglike_curr[inds]
        logprior_curr = logprior_curr[inds]

        log_W = -log(N)*ones(N)
    
        # MCMC Diversify step
        for i in 1:N
            for k in 1:R
                ξ_prop = rand(MvNormal(ξ[i, :], Σ))
                loglike_prop = loglike(ξ_prop)
                logprior_prop = logprior(ξ_prop)
                log_rmh = γ[t] * (loglike_prop - loglike_curr[i]) + logprior_prop - logprior_curr[i]

                # Accept the proposal with probability RMH
                if rand() < exp(log_rmh)
                    ξ[i, :] = ξ_prop
                    loglike_curr[i] = loglike_prop
                    logprior_curr[i] = logprior_prop
                end
            end
        end
        println("Current γ = ",γ[t])
        println("Unique particles: ",length(unique(ξ[:, 1])))
    end
    return ξ
end

**NOTE**: this code is also replicated in the `smc_sampler_lanneal.jl`

## Exercises

Some exercises for solidifying knowledge:

1. Adjust the code to implement the adaptive version of the sampler: **Continue reading this document for a primer** 
2. Read and implement the adaptive SMC sampler for the predator-prey model: `lv_smc_example.ipynb`
3. Try running the SMC sampler using the package SequentialMonteCarlo.jl, see the `lvsmc` folder

## Adaptive version primer

### Algorithm

Input: Number MCMC steps $R$.

1. Draw $\xi_1^i \sim \pi_1(\theta)$ for $i=1,\ldots,N$. 
2. While $\gamma_t \neq 1$
    - (i) If ESS($\gamma = 1)>N/2$ set $\gamma_{t} = 1$ otherwise find $\gamma_{t}$ such that ESS($\gamma$) $\approx N/2$.
    - (ii) Calculate weights 
        $$
        \tilde{w}_{t}^i = W_{t-1}^if(y|\theta)^{\gamma_t - \gamma_{t-1}} \qquad W_{t}^i = \frac{\tilde{w}_{t}^i}{\sum_{j=1}^N\tilde{w}_{t}^j}, \text{  for  } i=1,\ldots,N.
        $$
    - (iii) Resample $A_t^i$~Cat $(W_{t}^1,...,W_{t}^N)$ and set $W_t^i=1/N$ for $i\in \{1,...,N\}$
    - (iv) Compute $\hat{\Sigma}$ as the sample covariance of the particles $\{\xi^{A_t^i}_{t-1}\}_{i=1}^N$.
    - (v) Diversify using $R$ MCMC iterations with proposal covariance $\hat{\Sigma}$
        $$
        \xi_{t}^i \sim K_t(\theta_t \mid \theta_{t-1} = \xi_{t-1}^{A_{t}^i}) , \text{  for  } i=1,\ldots,N.
        $$


### Create some utility functions

Recall the weights for target $t$ are given by
\begin{align*}
W_t^i \propto f(y|\theta = \xi_{t-1}^i)^{\gamma_t - \gamma_{t-1}}
\end{align*}
We know that ESS = $1/\sum_{i=1}^N (W_t^i)^2$.  

Create a function compute_ess_diff that returns $\text{ESS}(\gamma_t) - N/2$ given $\gamma_{t-1}$ and the log_like values of the particles $\xi_{t-1}^i$. 


In [ ]:
# Illustration purpose only
function compute_ess_diff(γ, γ_old, loglike_curr)
    N = length(log_like)
    log_propto_W = ...
    log_propto_W = ... # Normalising step
    return exp(-logsumexp(2.0 * log_propto_W)) - N /2
end

Once this is defined, you may also need the Roots package to use the bisection method. 

In [ ]:
# Illustration 
fn = x -> compute_ess_diff(x, γ_old, loglike_curr) # Define as a function of γ
γ = find_zero(fn, (γ_old, 1.0)) # Find the γ which is the root of fn

The solution may be found in the `smc_sampler_lanneal_adaptive.jl` file. 